In [ ]:
from IPython.display import display, Image
import cv2
from datetime import datetime
from confluent_kafka import Producer
import json
import os
import time

# Callback function được gọi khi Kafka Producer gửi tin nhắn thành công hoặc thất bại
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

# Khởi tạo cấu hình cho Kafka Producer
kafka_bootstrap_servers = 'localhost:9092'
kafka_topic = 'videodetect'
producer_config = {
    'bootstrap.servers': kafka_bootstrap_servers,
}
producer = Producer(producer_config)

# Đường dẫn của video đầu vào
cap = cv2.VideoCapture("C:\\Linhtinh\\Video\\VIRAT_S_000002.mp4")

# Thư mục để lưu trữ hình ảnh xuất
output_folder = "C:\\Users\\admin\\OneDrive\\saved_images"

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Đọc frame đầu tiên từ video
ret, frame1 = cap.read()
ret, frame2 = cap.read()
print(frame1.shape)

# Tạo cửa sổ mới để hiển thị video
cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Video", 800, 600)

# Vòng lặp chính để xử lý video frame by frame
while cap.isOpened():
    # Tính toán sự thay đổi giữa hai frame liên tiếp
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Xử lý các contours để xác định vùng di chuyển
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 900:
            continue
        # Vẽ hộp giới hạn xung quanh vùng di chuyển
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: Movement", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        # Hiển thị video trong cửa sổ mới
        cv2.imshow("Video", frame1)

        # Hiển thị hình ảnh trên console Jupyter
        _, img_encoded = cv2.imencode('.jpg', frame1)
        display(Image(data=img_encoded.tobytes(), format='jpg'))

        # Tạo một dictionary chứa thông tin về sự kiện
        event_datetime = datetime.now()
        event_data = {
            "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
            "datetime": event_datetime.strftime("%Y-%m-%d %H:%M:%S")
        }

        # Chuyển đổi dictionary thành chuỗi JSON
        event_json = json.dumps(event_data)

        # In dữ liệu JSON ra console
        print(event_json)

        # Gửi dữ liệu JSON đến Kafka
        producer.produce(kafka_topic, key=None, value=event_json, callback=delivery_report)

        # Lưu hình ảnh vào thư mục xuất
        frame_filename = f"{output_folder}\\frame_{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}.jpg"
        cv2.imwrite(frame_filename, frame1)

    frame1 = frame2
    ret, frame2 = cap.read()

    # Dừng vòng lặp nếu nhấn phím Esc
    if cv2.waitKey(40) == 27:
        break

    # Chờ 1 giây trước khi lặp lại
    time.sleep(1)

# Đóng cửa sổ video và giải phóng tài nguyên khi kết thúc
cv2.destroyAllWindows()
cap.release()
